## ANALYSE D'INDICATEURS SUR LA VIE SCOLAIRE ET EXTRA-SCOLAIRE DES ÉLÈVES DE L'INSA TOULOUSE

### I. Objectif

Avec pour objectif d'aider tous les étudiants admis à obtenir leur diplôme quelle que soit leur origine sociale ou géographique, ou leur parcours scolaire avant d'entrer à l'INSA. Nous souhaitons donc pouvoir connaître les facteurs qui peuvent affecter la capacité de réussitte des étudiants afin de pouvoir trouver des solutions pour améliorer le taux de réussite des étudiants de l'INSA Toulouse.

### II. Traitement de données

Nous avons les données collectées d'une enquête réalisée par des étudiants de l'INSA Toulouse en mai 2024, effectuée via l'application Limesurvey. Ensuite, nous avons un fichier Excel contenant des lignes représentant chaque étudiant et des colonnes correspondant aux réponses. Cependant, comme certaines questions sont des QCM, les réponses d'une question ne se trouvent pas dans une seule colonne mais dans plusieurs colonnes. Par conséquent, nous devons transformer ces colonnes en une seule colonne.

In [7]:
pip install fuzzywuzzy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from fuzzywuzzy import process

# Load the Excel file
file_path = './results-survey278778.xlsx'
data = pd.read_excel(file_path)

# Define the function to combine multiple choice columns into a single column
def combine_choices(row, choice_cols, choice_labels):
    for col, label in zip(choice_cols, choice_labels):
        if row[col] == 'Oui':
            return label
    return 'No response'

# Function to handle 'other' responses
def handle_other_response(open_response, choices):
    best_match, score = process.extractOne(open_response, choices)
    if score > 80:  # threshold for a good match
        return best_match
    return 'Other: ' + open_response

# Define questions mapping
questions_mapping = {

    'LOGEMENTTYPE': {
        'columns': ['LOGEMENTTYPE[SQ001]', 'LOGEMENTTYPE[SQ002]', 'LOGEMENTTYPE[SQ003]', 'LOGEMENTTYPE[other]'],
        'choices': ['Seul', 'Colocation', 'Foyer familial', 'Autre']
    },
    }


# Process each question and create new combined columns
for question, mapping in questions_mapping.items():
    data[question] = data.apply(combine_choices, choice_cols=mapping['columns'], choice_labels=mapping['choices'], axis=1)

# Handle 'other' responses if any
for question, mapping in questions_mapping.items():
    if 'Other' in mapping['columns']:
        other_column = f'{question}[other]'
        if other_column in data.columns:
            data[question] = data.apply(lambda row: handle_other_response(row[other_column], mapping['choices']) 
                                        if row[question] == 'Autre' else row[question], axis=1)


